In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['model-num', 'hindienglish-corpora']


# Generate the dataset

In [40]:
import random
en_hi_nums = {
    "one": "एक",
    "two": "दो",
    "three": "तीन",
    "four": "चार",
    "five": "पांच",
    "six": "छः",
    "seven": "सात",
    "eight": "आठ",
    "nine": "नौ",
    "zero": "शून्य"
}

lines = pd.DataFrame({"english_sentence": [], "hindi_sentence": []})
for i in range(15000):
    rand_range = random.randint(1, 26)
    en_sent = ''
    hi_sent = 'START_'
    for i in range(rand_range):
        dig = random.choice(list(en_hi_nums.keys()))
        en_sent = en_sent + ' ' + dig
        hi_sent = hi_sent + ' ' + en_hi_nums[dig]
    hi_sent = hi_sent + ' ' + ' _END'
    lines = lines.append({"english_sentence": en_sent, "hindi_sentence": hi_sent}, ignore_index=True)

In [41]:
print(len(lines))
lines.head()

15000


,english_sentence,hindi_sentence
0,two zero nine seven six nine four eight three five nine five two three one nine five,START_ दो शून्य नौ सात छः नौ चार आठ तीन पांच नौ पांच दो तीन एक नौ पांच _END
1,one one two nine five five eight eight,START_ एक एक दो नौ पांच पांच आठ आठ _END
2,eight four zero three three seven six nine zero six eight,START_ आठ चार शून्य तीन तीन सात छः नौ शून्य छः आठ _END
3,five three four,START_ पांच तीन चार _END
4,seven five eight six five three five four one four zero one three eight zero five nine eight five nine nine one seven nine nine eight,START_ सात पांच आठ छः पांच तीन पांच चार एक चार शून्य एक तीन आठ शून्य पांच नौ आठ पांच नौ नौ एक सात नौ नौ आठ _END


In [42]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [43]:
len(all_eng_words)

10

In [44]:
len(all_hindi_words)

12

In [45]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [46]:
lines.head()

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
0,two zero nine seven six nine four eight three five nine five two three one nine five,START_ दो शून्य नौ सात छः नौ चार आठ तीन पांच नौ पांच दो तीन एक नौ पांच _END,18,20
1,one one two nine five five eight eight,START_ एक एक दो नौ पांच पांच आठ आठ _END,9,11
2,eight four zero three three seven six nine zero six eight,START_ आठ चार शून्य तीन तीन सात छः नौ शून्य छः आठ _END,12,14
3,five three four,START_ पांच तीन चार _END,4,6
4,seven five eight six five three five four one four zero one three eight zero five nine eight five nine nine one seven nine nine eight,START_ सात पांच आठ छः पांच तीन पांच चार एक चार शून्य एक तीन आठ शून्य पांच नौ आठ पांच नौ नौ एक सात नौ नौ आठ _END,27,29


In [47]:
lines[lines['length_eng_sentence']>30].shape

(0, 4)

In [48]:
# lines=lines[lines['length_eng_sentence']<=20]
# lines=lines[lines['length_hin_sentence']<=20]

In [49]:
lines.shape

(15000, 4)

In [50]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  29
maximum length of English Sentence  27


In [51]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [52]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(10, 12)

In [54]:
num_decoder_tokens += 1 #for zero padding

In [55]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [56]:
input_token_index, target_token_index

({'eight': 1,
  'five': 2,
  'four': 3,
  'nine': 4,
  'one': 5,
  'seven': 6,
  'six': 7,
  'three': 8,
  'two': 9,
  'zero': 10},
 {'START_': 1,
  '_END': 2,
  'आठ': 3,
  'एक': 4,
  'चार': 5,
  'छः': 6,
  'तीन': 7,
  'दो': 8,
  'नौ': 9,
  'पांच': 10,
  'शून्य': 11,
  'सात': 12})

In [57]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [58]:
lines = shuffle(lines)
lines.head(10)

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
12063,four eight six nine nine zero one three,START_ चार आठ छः नौ नौ शून्य एक तीन _END,9,11
5928,one three zero three seven six seven nine,START_ एक तीन शून्य तीन सात छः सात नौ _END,9,11
182,eight nine zero six seven zero eight seven two two nine six eight two three zero,START_ आठ नौ शून्य छः सात शून्य आठ सात दो दो नौ छः आठ दो तीन शून्य _END,17,19
8730,zero four one seven eight nine seven four zero,START_ शून्य चार एक सात आठ नौ सात चार शून्य _END,10,12
3178,six,START_ छः _END,2,4
2596,three three seven eight,START_ तीन तीन सात आठ _END,5,7
6959,six nine seven four three five five nine one zero five seven four five six one six seven zero three,START_ छः नौ सात चार तीन पांच पांच नौ एक शून्य पांच सात चार पांच छः एक छः सात शून्य तीन _END,21,23
11493,zero four three four two two five three four three six seven five zero eight,START_ शून्य चार तीन चार दो दो पांच तीन चार तीन छः सात पांच शून्य आठ _END,16,18
11718,seven five four zero three nine zero one seven nine three zero two nine five nine two,START_ सात पांच चार शून्य तीन नौ शून्य एक सात नौ तीन शून्य दो नौ पांच नौ दो _END,18,20
7343,nine six nine five two nine zero six nine nine six three five eight zero,START_ नौ छः नौ पांच दो नौ शून्य छः नौ नौ छः तीन पांच आठ शून्य _END,16,18


### Split the data into train and test

In [59]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((12000,), (3000,))

In [60]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [61]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [62]:
latent_dim=200

In [63]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [64]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [65]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [66]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    2000        input_5[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 200)    2600        input_6[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LS

In [67]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

## Train the model

In [68]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
93/93 [==============================] - 9s 98ms/step - loss: 2.3307 - val_loss: 2.2322
Epoch 2/50
93/93 [==============================] - 8s 82ms/step - loss: 2.1570 - val_loss: 2.0109
Epoch 3/50
93/93 [==============================] - 7s 78ms/step - loss: 1.9461 - val_loss: 1.8067
Epoch 4/50
93/93 [==============================] - 8s 87ms/step - loss: 1.7108 - val_loss: 1.6344
Epoch 5/50
93/93 [==============================] - 8s 81ms/step - loss: 1.5574 - val_loss: 1.4516
Epoch 6/50
93/93 [==============================] - 7s 80ms/step - loss: 1.3848 - val_loss: 1.2236
Epoch 7/50
93/93 [==============================] - 7s 80ms/step - loss: 1.2381 - val_loss: 1.2001
Epoch 8/50
93/93 [==============================] - 8s 87ms/step - loss: 1.1368 - val_loss: 1.0165
Epoch 9/50
93/93 [==============================] - 8s 82ms/step - loss: 1.0503 - 

In [69]:
model.save_weights('nmt_weights_num_data_working_2.h5')

In [35]:
# model.load_weights('../input/model-num/model/nmt_weights_num_data_working.h5')

In [70]:

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [71]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Sample usage

In [97]:
inp_seq = "one one one two two three"
tmp_is = inp_seq
inp_seq = np.array([[input_token_index[x] for x in inp_seq.split()]], dtype=np.float)
# print(inp_seq.shape)
decoded_sentence = decode_sequence(inp_seq)
print(inp_seq)
print(tmp_is)
print(decoded_sentence)

[[5. 5. 5. 9. 9. 8.]]
one one one two two three
 एक एक दो एक दो तीन _END


In [87]:
asr_res_df = pd.read_csv('../input/asr-res/test_res.csv', usecols=['file', 'transcript'])

## Run inference on ASR output

In [91]:
totext = {
    "0": "zero",
    "1": "one",
    "2": "two",
    "3": "three",
    "4": "four",
    "5": "five",
    "6": "six",
    "7": "seven",
    "8": "eight",
    "9": "nine",
    "O": "zero",
    "Z": "zero"
}

def gen_hindi_tg(fname):
    ts = fname.split('_')[-1][:-1]
    en_tg = [totext[x] for x in ts]
    hi_tg = ' '.join(en_hi_nums[x] for x in en_tg)
    return hi_tg

In [94]:
asr_res_df['hindi_target'] = asr_res_df.apply(lambda row: gen_hindi_tg(row['file']), axis=1)

In [95]:
asr_res_df.head()

,file,transcript,hindi_target
0,MAN_AH_100_416A,FOUR ONE SIX,चार एक छः
1,MAN_AH_101_428OOO9A,FOUR TWO EIGHT ZERO ZERO ZERO NINE,चार दो आठ शून्य शून्य शून्य नौ
2,MAN_AH_102_452Z3A,FOUR FIVE TWO ZERO THREE,चार पांच दो शून्य तीन
3,MAN_AH_103_4625A,FOUR SIX TWO FIVE,चार छः दो पांच
4,MAN_AH_104_4A,FOUR,चार


In [99]:
def tr(inp_seq):
    inp_seq = np.array([[input_token_index[x] for x in inp_seq.split()]], dtype=np.float)
    decoded_sentence = decode_sequence(inp_seq)
    return decoded_sentence

asr_res_df['hindi_translated'] = asr_res_df.apply(lambda row: tr(row['transcript'].lower()), axis=1)

In [100]:
asr_res_df.head()

,file,transcript,hindi_target,hindi_translated
0,MAN_AH_100_416A,FOUR ONE SIX,चार एक छः,चार एक छः _END
1,MAN_AH_101_428OOO9A,FOUR TWO EIGHT ZERO ZERO ZERO NINE,चार दो आठ शून्य शून्य शून्य नौ,चार दो आठ शून्य शून्य शून्य नौ _END
2,MAN_AH_102_452Z3A,FOUR FIVE TWO ZERO THREE,चार पांच दो शून्य तीन,चार पांच दो शून्य तीन _END
3,MAN_AH_103_4625A,FOUR SIX TWO FIVE,चार छः दो पांच,चार छः दो पांच _END
4,MAN_AH_104_4A,FOUR,चार,चार _END


In [102]:
asr_res_df.to_csv("mt_res.csv")